In [5]:
from pandas import read_csv

from data_process import prepare_data

train_df = prepare_data('./data/train.csv')
test_df = prepare_data('./data/test.csv')
actual_df = read_csv('./data/gender_submission.csv')

Sex: ['female' 'male']
Embarked: ['C' 'Q' 'S' 'nan']
Sex: ['female' 'male']
Embarked: ['C' 'Q' 'S']


In [2]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

parameters={
    # 'n_estimators': [30, 35, 40, 45],
    'min_samples_leaf': [2, 3, 4, 5, 6, 7],
    'max_features': [4, 5, 6, 7]
}

grid_search_cv = GridSearchCV(RandomForestClassifier(n_jobs=-1, random_state=42, n_estimators=40), parameters, n_jobs=-1)

X_train = train_df.drop(['Survived'], axis=1)
Y_train = train_df['Survived']

grid_search_cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'min_samples_leaf': [2, 3, 4, 5, 6, 7], 'max_features': [4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [7]:
from sklearn.metrics import mean_absolute_error

not_na = test_df.notna().all(axis=1)

X_test = test_df[not_na]
Y_test = actual_df[not_na]['Survived']


0.11750599520383694

In [14]:
Y_predict = grid_search_cv.predict(X_test)

mean_absolute_error(Y_test, Y_predict)

0.11031175059952038

In [15]:
grid_search_cv.best_params_

{'max_features': 7, 'min_samples_leaf': 5}

In [16]:
from sklearn_porter import Porter

porter = Porter(grid_search_cv.best_estimator_)

export = porter.export('TitanicClassifier')

with open('TitanicClassifier.java', 'w') as file:
    file.write(export)